In [1]:
import sys
sys.path.append('../')
import corecomposition as cc

import numpy as np
import pyvo as vo
import matplotlib.pyplot as plt
from astropy.table import Table, join

In [2]:
gold = Table.read('../data/gold.csv')
silver = Table.read('../data/silver.csv')
bronze = Table.read('../data/bronze.csv')

In [3]:
photo = cc.photometry.fetch_photometry(bronze['wd_source_id'])
bronze = join(bronze, photo, keys_left='wd_source_id', keys_right='source_id')


In [4]:
photo[photo['source_id'] == 1307515077152381568]

source_id,PS1_g,PS1_r,PS1_i,PS1_z,PS1_y,e_PS1_g,e_PS1_r,e_PS1_i,e_PS1_z,e_PS1_y,source,qualityFlag
,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,
int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str3,uint8
1307515077152381568,20.4947,20.3694,20.0103,19.7876,-999.0,0.026247,0.033681,0.044099,0.108458,-999.0,ps1,53


In [5]:
source_ids = bronze['wd_source_id']
distances = bronze['distance']
astrometric_params = bronze['wd_astrometric_params_solved']
photo = np.array([bronze['wd_phot_g_mean_mag'], bronze['wd_phot_bp_mean_mag'], bronze['wd_phot_rp_mean_mag']]).T
e_photo = np.array([bronze['wd_e_gmag'], bronze['wd_e_bpmag'], bronze['wd_e_rpmag']]).T
initial_params = np.array([bronze['CO_Hdef_teff'], bronze['CO_Hdef_radius']]).T

In [6]:
photo[0]

array([20.463974  , 20.26536369, 20.02454376])

In [7]:
i = 0
obj = cc.photometry.Photometry(source_ids, distances, astrometric_params, photo, e_photo, initial_params)

In [8]:
obj.photo[1307515077152381568]

[191.31002807617188,
 31,
 array(['Gaia_G', 'Gaia_BP', 'Gaia_RP', 'PS1_g', 'PS1_r', 'PS1_i', 'PS1_z',
        'PS1_y'], dtype='<U7'),
 array([  20.463974  ,   20.26536369,   20.02454376,   20.49469948,
          20.36940002,   20.01029968,   19.78759956, -999.        ]),
 array([ 4.66615165e-03,  1.41416389e-01,  9.53912634e-02,  2.62470003e-02,
         3.36810015e-02,  4.40989994e-02,  1.08457997e-01, -9.99000000e+02]),
 array([8.89800454e+03, 7.23204103e-03])]

In [9]:
obj.run_mcmc(cc.interpolator.LaPlataBase, layer='Hrich')

TypeError: Photometry.run_mcmc() missing 1 required positional argument: 'initial_params'

In [ ]:
gold_coverage = join(gold, table, keys_left='wd_source_id', keys_right='source_id')
silver_coverage = join(silver, table, keys_left='wd_source_id', keys_right='source_id')
bronze_coverage = table

print(f'gold coverage = {len(gold_coverage)} / {len(gold)}')
print(f'silver coverage = {len(silver_coverage)} / {len(silver)}')
print(f'bronze coverage = {len(bronze_coverage)} / {len(bronze)}')

NameError: name 'table' is not defined

In [ ]:
gold_coverage

wd_solution_id,ms_solution_id,wd_source_id,ms_source_id,wd_random_index,ms_random_index,wd_ref_epoch,ms_ref_epoch,wd_ra,ms_ra,wd_ra_error,ms_ra_error,wd_dec,ms_dec,wd_dec_error,ms_dec_error,wd_parallax,ms_parallax,wd_parallax_error,ms_parallax_error,wd_parallax_over_error,ms_parallax_over_error,wd_pm,ms_pm,wd_pmra,ms_pmra,wd_pmra_error,ms_pmra_error,wd_pmdec,ms_pmdec,wd_pmdec_error,ms_pmdec_error,wd_ra_dec_corr,ms_ra_dec_corr,wd_ra_parallax_corr,ms_ra_parallax_corr,wd_ra_pmra_corr,ms_ra_pmra_corr,wd_ra_pmdec_corr,ms_ra_pmdec_corr,wd_dec_parallax_corr,ms_dec_parallax_corr,wd_dec_pmra_corr,ms_dec_pmra_corr,wd_dec_pmdec_corr,ms_dec_pmdec_corr,wd_parallax_pmra_corr,ms_parallax_pmra_corr,wd_parallax_pmdec_corr,ms_parallax_pmdec_corr,wd_pmra_pmdec_corr,ms_pmra_pmdec_corr,wd_astrometric_n_obs_al,ms_astrometric_n_obs_al,wd_astrometric_n_obs_ac,ms_astrometric_n_obs_ac,wd_astrometric_n_good_obs_al,ms_astrometric_n_good_obs_al,wd_astrometric_n_bad_obs_al,ms_astrometric_n_bad_obs_al,wd_astrometric_gof_al,ms_astrometric_gof_al,wd_astrometric_chi2_al,ms_astrometric_chi2_al,wd_astrometric_excess_noise,ms_astrometric_excess_noise,wd_astrometric_excess_noise_sig,ms_astrometric_excess_noise_sig,wd_astrometric_params_solved,ms_astrometric_params_solved,wd_astrometric_primary_flag,ms_astrometric_primary_flag,wd_nu_eff_used_in_astrometry,ms_nu_eff_used_in_astrometry,wd_pseudocolour,ms_pseudocolour,wd_pseudocolour_error,ms_pseudocolour_error,wd_ra_pseudocolour_corr,ms_ra_pseudocolour_corr,wd_dec_pseudocolour_corr,ms_dec_pseudocolour_corr,wd_parallax_pseudocolour_corr,ms_parallax_pseudocolour_corr,wd_pmra_pseudocolour_corr,ms_pmra_pseudocolour_corr,wd_pmdec_pseudocolour_corr,ms_pmdec_pseudocolour_corr,wd_astrometric_matched_transits,ms_astrometric_matched_transits,wd_visibility_periods_used,ms_visibility_periods_used,wd_astrometric_sigma5d_max,ms_astrometric_sigma5d_max,wd_matched_transits,ms_matched_transits,wd_new_matched_transits,ms_new_matched_transits,wd_matched_transits_removed,ms_matched_transits_removed,wd_ipd_gof_harmonic_amplitude,ms_ipd_gof_harmonic_amplitude,wd_ipd_gof_harmonic_phase,ms_ipd_gof_harmonic_phase,wd_ipd_frac_multi_peak,ms_ipd_frac_multi_peak,wd_ipd_frac_odd_win,ms_ipd_frac_odd_win,wd_ruwe,ms_ruwe,wd_scan_direction_strength_k1,ms_scan_direction_strength_k1,wd_scan_direction_strength_k2,ms_scan_direction_strength_k2,wd_scan_direction_strength_k3,ms_scan_direction_strength_k3,wd_scan_direction_strength_k4,ms_scan_direction_strength_k4,wd_scan_direction_mean_k1,ms_scan_direction_mean_k1,wd_scan_direction_mean_k2,ms_scan_direction_mean_k2,wd_scan_direction_mean_k3,ms_scan_direction_mean_k3,wd_scan_direction_mean_k4,ms_scan_direction_mean_k4,wd_duplicated_source,ms_duplicated_source,wd_phot_g_n_obs,ms_phot_g_n_obs,wd_phot_g_mean_flux,ms_phot_g_mean_flux,wd_phot_g_mean_flux_error,ms_phot_g_mean_flux_error,wd_phot_g_mean_flux_over_error,ms_phot_g_mean_flux_over_error,wd_phot_g_mean_mag,ms_phot_g_mean_mag,wd_phot_bp_n_obs,ms_phot_bp_n_obs,wd_phot_bp_mean_flux,ms_phot_bp_mean_flux,wd_phot_bp_mean_flux_error,ms_phot_bp_mean_flux_error,wd_phot_bp_mean_flux_over_error,ms_phot_bp_mean_flux_over_error,wd_phot_bp_mean_mag,ms_phot_bp_mean_mag,wd_phot_rp_n_obs,ms_phot_rp_n_obs,wd_phot_rp_mean_flux,ms_phot_rp_mean_flux,wd_phot_rp_mean_flux_error,ms_phot_rp_mean_flux_error,wd_phot_rp_mean_flux_over_error,ms_phot_rp_mean_flux_over_error,wd_phot_rp_mean_mag,ms_phot_rp_mean_mag,wd_phot_bp_n_contaminated_transits,ms_phot_bp_n_contaminated_transits,wd_phot_bp_n_blended_transits,ms_phot_bp_n_blended_transits,wd_phot_rp_n_contaminated_transits,ms_phot_rp_n_contaminated_transits,wd_phot_rp_n_blended_transits,ms_phot_rp_n_blended_transits,wd_phot_proc_mode,ms_phot_proc_mode,wd_phot_bp_rp_excess_factor,ms_phot_bp_rp_excess_factor,wd_bp_rp,ms_bp_rp,wd_bp_g,ms_bp_g,wd_g_rp,ms_g_rp,wd_dr2_radial_velocity,ms_dr2_radial_velocity,wd_dr2_radial_velocity_error,ms_dr2_radial_velocity_error,wd_dr2_rv_nb_transits,ms_dr2_rv_nb_transits,wd_dr2_rv_template_teff,ms_dr2